<span style="font-size:36px"><b>Baseline Model Tutorial</b></span>

Copyright &copy; 2020 Gunawan Lumban Gaol

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language overning permissions and limitations under the License.

# Import Packages

In [ ]:
import os
import sys
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from ipywidgets import interact
import tensorflow as tf
import tensorflow.keras.backend as K
print("tf.__version__ = "+tf.__version__)

from gurih.data.data_generator import DataGenerator, get_y_true_data_generator, validate_dataset_dir
from gurih.models.decoder import CTCDecoder
from gurih.models.model import BaselineASRModel
from gurih.models.utils import CharMap, wer

%load_ext autoreload
%autoreload 2

Set documentation directory and model weights directory.

In [ ]:
model_dir = "../../models/Model010b/"  # to store model weights
doc_dir = "../../docs/Model010b/"  # to store model configuration and evaluation

if not os.path.exists(model_dir): os.makedirs(model_dir)
if not os.path.exists(doc_dir): os.makedirs(doc_dir)

Also specify `train`, `validation`, and `test` dataset directories. 

These directories should contain all `.npz` and its corresponding `.txt` transcriptions files with same naming conventions. For example if npz filename is `0.npz`, then the transcription txt filename should be `0.txt`.

In [ ]:
train_dir = "../../dataset/interim/Model-010b/train/"
valid_dir = "../../dataset/interim/Model-010b/valid/"

In [ ]:
validate_dataset_dir(train_dir)
validate_dataset_dir(valid_dir)

# Model Training

Define constant for ASR Model.

In [ ]:
CHAR_TO_IDX_MAP = CharMap.CHAR_TO_IDX_MAP
IDX_TO_CHAR_MAP = CharMap.IDX_TO_CHAR_MAP

MAX_SEQ_LENGTH = 3000
MAX_LABEL_LENGTH = 300
BATCH_SIZE = 16
MAX_EPOCHS = 5000

Create and compile keras model.

In [ ]:
BaselineASR = BaselineASRModel(input_shape=(MAX_SEQ_LENGTH, 39), vocab_len=len(CharMap()), dir_path=model_dir, doc_path=doc_dir)
BaselineASR.compile()

Get model output sequence length for ctc input. This is required as we are using custom `Lambda` layer inside model.

In [ ]:
CTC_INPUT_LENGTH = BaselineASR.model.get_layer('the_output').output.shape[1]
CTC_INPUT_LENGTH

## Train with Generator

Prepare data using `DataGenerator`. Directories must contain all `.npz` and `.txt` files required for `DataGenerator`.

In [ ]:
train_generator = DataGenerator(input_dir=train_dir,
                                max_seq_length=MAX_SEQ_LENGTH,
                                max_label_length=MAX_LABEL_LENGTH,
                                ctc_input_length=CTC_INPUT_LENGTH,
                                char_to_idx_map=CHAR_TO_IDX_MAP,
                                batch_size=BATCH_SIZE)

validation_generator = DataGenerator(input_dir=valid_dir,
                                     max_seq_length=MAX_SEQ_LENGTH,
                                     max_label_length=MAX_LABEL_LENGTH,
                                     ctc_input_length=CTC_INPUT_LENGTH,
                                     char_to_idx_map=CHAR_TO_IDX_MAP,
                                     batch_size=BATCH_SIZE)

Pass validation generator if using `EarlyStopping` and `ModelCheckpoint` callbacks. The default `_callbacks()` method also use `CSVLogger` to write epochs output in csv file based on the documentation directory of model.

In [ ]:
BaselineASR._callbacks(min_delta=1e-4, patience=5, save_weights_only=True)

# Optionally, write your custom callbacks here
# BaselineASR.callbacks = []

Optionally, retrained pretrained model by loading weights.

In [ ]:
# filename = os.path.abspath("../../models/BaselineASR_f200_k11_s2_pvalid_nlstm200_ndense29.h5") # must provide abs path
# BaselineASR.load(filename)

In [ ]:
BaselineASR.fit_generator(train_generator=train_generator,
                          validation_generator=validation_generator,
                          epochs=MAX_EPOCHS,
#                           use_multiprocessing=False,
#                           worker=4,
)

See and save training plots.

In [ ]:
# If not retraining, history will not be available in the model.
# BaselineASR.plot_history()

# To plot using csv available from CSVLogger callback
df_hist = pd.read_csv(doc_dir+'BaselineASR_f200_k11_s2_pvalid_nlstm200_ndense29.csv')
df_hist.set_index('epoch', inplace=True)
df_hist.plot(figsize=(15, 6))
plt.savefig(doc_dir+'BaselineASR_f200_k11_s2_pvalid_nlstm200_ndense29.png', dpi=300)
plt.show()

Additionally, you may want to serialize both the model and weights.

In [ ]:
# BaselineASR.save()

# Model Evaluation

Evaluate model on validation dataset.

In [ ]:
# model = BaselineASR

model = BaselineASRModel(input_shape=(MAX_SEQ_LENGTH, 39), vocab_len=len(CharMap()), dir_path=model_dir, doc_path=doc_dir)
model.compile()
model.load(os.path.abspath(model_dir+"BaselineASR_f200_k11_s2_pvalid_nlstm200_ndense29.h5")) # must provide abs path

Prepare the dataset for evaluation.

In [ ]:
train_generator = DataGenerator(input_dir=train_dir,
                                max_seq_length=MAX_SEQ_LENGTH,
                                max_label_length=MAX_LABEL_LENGTH,
                                ctc_input_length=CTC_INPUT_LENGTH,
                                char_to_idx_map=CHAR_TO_IDX_MAP,
                                batch_size=BATCH_SIZE)

validation_generator = DataGenerator(input_dir=valid_dir,
                                     max_seq_length=MAX_SEQ_LENGTH,
                                     max_label_length=MAX_LABEL_LENGTH,
                                     ctc_input_length=CTC_INPUT_LENGTH,
                                     char_to_idx_map=CHAR_TO_IDX_MAP,
                                     batch_size=BATCH_SIZE)

## Loss & WER Evaluation

Calculate model `loss` and `WER` for train and validation dataset.

In [ ]:
train_loss, train_ctc_matrix = model.evaluate(train_generator, low_memory=True)
val_loss, val_ctc_matrix = model.evaluate(validation_generator, low_memory=True)

In [ ]:
print(f"Train\tloss: {train_loss.mean():.3f}\tshape: {train_loss.shape}")
print(f"Valid\tloss: {val_loss.mean():.3f}\tshape: {val_loss.shape}")

In [ ]:
decoder = CTCDecoder(IDX_TO_CHAR_MAP)
decoder.fit(train_ctc_matrix)

In [ ]:
train_y_true = get_y_true_data_generator(IDX_TO_CHAR_MAP, train_generator)
train_y_preds = decoder.predict(train_ctc_matrix)
val_y_true = get_y_true_data_generator(IDX_TO_CHAR_MAP, validation_generator)
val_y_preds = decoder.predict(val_ctc_matrix)

In [ ]:
train_wer = wer(train_y_true, train_y_preds)
val_wer = wer(val_y_true, val_y_preds)

In [ ]:
print(f"Train\tloss: {train_loss.mean():.3f}\twer: {train_wer:.3f}\tshape: {train_loss.shape}")
print(f"Val\tloss: {val_loss.mean():.3f}\twer: {val_wer:.3f}\tshape: {val_loss.shape}")

Save evaluation metrics to `csv` files.

In [ ]:
metrics = [
    ['train', train_loss.mean(), train_wer, train_loss.shape[0]],
    ['val', val_loss.mean(), val_wer, val_loss.shape[0]]
]
df_metrics = pd.DataFrame(metrics, columns=['dataset', 'loss', 'wer', 'num'])
df_metrics.to_csv(doc_dir+'metrics.csv', index=False, line_terminator='\n', sep=',')

# Performance on Test Dataset

Evaluate using test data.

\*\***IMPORTANT NOTICE**\*\*: 

Doing this will render your test set __obsolete__ as you would've already seen the datatest set you're about to evaluate. Please make sure you have optimize `bias-variance` tradeoff of the model using `validation` dataset before doing this analysis.

Choose which model you want to evaluate.

In [ ]:
# model = BaselineASR

model = BaselineASRModel(input_shape=(MAX_SEQ_LENGTH, 39), vocab_len=len(CharMap()), dir_path=model_dir, doc_path=doc_dir)
model.compile()
model.load(os.path.abspath(model_dir+"BaselineASR_f200_k11_s2_pvalid_nlstm200_ndense29.h5")) # must provide abs path

## Loss & WER Evaluation

Evaluate overall `loss` calculated from test set. Use decoder to get prediction str of input sequence and calculate word error rate.

In [ ]:
test_dir = "../../dataset/interim/Model-010a/test/"
validate_dataset_dir(test_dir)

In [ ]:
test_generator = DataGenerator(input_dir=test_dir,
                               max_seq_length=MAX_SEQ_LENGTH,
                               max_label_length=MAX_LABEL_LENGTH,
                               ctc_input_length=CTC_INPUT_LENGTH,
                               char_to_idx_map=CHAR_TO_IDX_MAP,
                               batch_size=8,
                               shuffle=False) # set to False to ensure correct alignment between y_true and y_pred

In [ ]:
test_loss, test_ctc_matrix = model.evaluate(test_generator, low_memory=True)
decoder = CTCDecoder(IDX_TO_CHAR_MAP)
decoder.fit(test_ctc_matrix)

In [ ]:
y_true = get_y_true_data_generator(IDX_TO_CHAR_MAP, test_generator)
y_pred = decoder.predict(test_ctc_matrix)
test_wer = wer(y_true, y_pred)

In [ ]:
print(f"Test\tloss: {test_loss.mean():.3f}\twer: {test_wer:.3f}\tshape: {test_loss.shape}")

Save test metrics results. This will append to metrics files created in Model Evaluation step.

In [ ]:
test_metric = ['test', test_loss.mean(), test_wer, test_loss.shape[0]]

In [ ]:
df_metrics = pd.read_csv(doc_dir+'metrics.csv')
df_metrics.loc[len(df_metrics)] = test_metric
df_metrics.to_csv(doc_dir+'metrics.csv', index=False, line_terminator='\n', sep=',')

## Sample Trancription Evaluation

Perform deeper analysis on sample transcription output.

In [ ]:
sample_test_generator = DataGenerator(input_dir=test_dir,
                                      max_seq_length=MAX_SEQ_LENGTH,
                                      max_label_length=MAX_LABEL_LENGTH,
                                      ctc_input_length=CTC_INPUT_LENGTH,
                                      char_to_idx_map=CHAR_TO_IDX_MAP,
                                      batch_size=100000, # set large to load all or num examples you want from test_dir
                                      shuffle=False) # set to False to ensure correct alignment between y_true and y_pred

X_test = sample_test_generator[0][0]['the_input']
print(f"X_test shape: {X_test.shape}")

y_test = sample_test_generator[0][0]['the_labels']
y_test = np.array([''.join([IDX_TO_CHAR_MAP[c] for c in y]).strip() for y in y_test])
print(f"y_test shape: {y_test.shape}")

Get the `ctc_matrix`.

In [ ]:
ctc_matrix = model.predict(X_test)
decoder = CTCDecoder(IDX_TO_CHAR_MAP)
decoder.fit(ctc_matrix)

Get all predictions from ctc_matrix.

In [ ]:
y_preds = decoder.predict(ctc_matrix)
len(y_preds)

Calculate and display word error rate.

In [ ]:
idx_choice = 1
y_true = y_test[idx_choice]
y_pred = y_preds[idx_choice]

In [ ]:
print(y_true)
print()
print(y_pred)

In [ ]:
word_error_rate = wer(y_true, y_pred, write_html=True)
print('WER %.4f %%' % word_error_rate)

Display prediction errors. 
* Red colors are <span style="background-color:rgba(255, 0, 0, 0.23)">missing word</span>
* Green colors are <span style="background-color:rgba(0, 128, 0, 0.23)">insertion word</span>.
* Yellow colors are <span style="background-color:rgba(255, 165, 0, 0.5)">substitution word</span> and the references are in parenthesis.

In [ ]:
with open("diff.html", 'r') as f:
    html = f.readlines()[0]
display(HTML(html))